In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="mFFA0OPQrCWnnk7kRawr")
project = rf.workspace("joaoissamu").project("objectdetect-fs9hx")
version = project.version(1)
dataset = version.download("yolov5")



In [ ]:
pwd

In [ ]:
import os

if not os.path.exists('content/yolov5'):
  !git clone https://github.com/ultralytics/yolov5
  %cd yolov5
  !pip install -r requirements.txt # install dependencies

In [ ]:
%cat /content/ObjectDetect-1/data.yaml

In [ ]:
!python /content/yolov5/train.py --batch 16 --epochs 100 --data "/content/ObjectDetect-1/data.yaml" --cfg /content/yolov5/models/yolov5s.yaml  --weights yolov5s.pt --name test_yolov5s_100 --device 0

In [ ]:
from IPython.display import Image

Image(filename='/content/yolov5/runs/train/test_yolov5s_1002/train_batch2.jpg', width=900)

In [ ]:
import glob

var_img_list = glob.glob('/content/ObjectDetect-1/valid/images/*')

from IPython.display import Image
import os

val_img_path = var_img_list[0]
print(val_img_path)

!python detect.py --weights /content/yolov5/runs/train/test_yolov5s_1002/weights/best.pt --img 416 --conf 0.5 --exist-ok --source /content/ObjectDetect-1/valid/images/054376_jpg.rf.89624a42c6101480f64b20371c70a8fe.jpg

Image(os.path.join('/content/ObjectDetect-1/valid/images/', os.path.basename(val_img_path)))

In [ ]:
import torch
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import numpy as np
from google.colab.patches import cv2_imshow

model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/yolov5/runs/train/test_yolov5s_1002/weights/best.pt')

img = Image.open('/content/ObjectDetect-1/valid/images/054376_jpg.rf.89624a42c6101480f64b20371c70a8fe.jpg')

results = model(img)

In [ ]:
df = results.pandas().xyxy[0]

for index, row in df.iterrows():
    x1, y1, x2, y2, confidence, class_id, name = row
    print(f"Object: {name}, Confidence: {confidence}")
    print(f"Bounding box coordinates: ({x1}, {y1}), ({x2}, {y2})")

In [ ]:
img_cv = cv2.imread('/content/ObjectDetect-1/valid/images/054376_jpg.rf.89624a42c6101480f64b20371c70a8fe.jpg')

for index, row in df.iterrows():
    x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])

    # 객체 영역에 블러 적용
    roi = img_cv[y1:y2, x1:x2]
    blurred_roi = cv2.GaussianBlur(roi, (99, 99), 0)
    img_cv[y1:y2, x1:x2] = blurred_roi

    # 바운딩 박스 및 텍스트 추가
    cv2.rectangle(img_cv, (x1, y1), (x2, y2), (255, 0, 0), 2)
    cv2.putText(img_cv, row['name'], (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

# 이미지 출력
cv2_imshow(img_cv)